<h3>监控级别</h3>

- 以下首先回顾可以监控的组件

<h4>Spark应用程序和作业</h4>

- 通过Spark UI和Spark日志获取监控报告，报告包括Spark应用程序的运行状态信息，例如RDD转换和查询计划的执行信息

<h4>JVM</h4>

- Spark在Java虚拟机（JVM）上运行执行器，监控虚拟机（VM）能够更好地理解代码的运行方式
- JVM自带一些监视工具
    - 用于跟踪堆栈的jstack
    - 用于创建堆转储的（heap-dumps）的jmap
    - 用于报告时序统计信息的jstat
    - 用于可视化JVM属性的jconsole
- 操作系统
    - CPU, 网络，I/O
- 集群
    - 也可以监视运行Spark应用程序的集群，YARN, Mesos或Standalone集群

<h3>要监视什么</h3>

- 驱动器和执行器进程
    - 应用程序的所有状态都会在驱动器进程上有所反映，需要确保它正确而稳定的运行
- 查询，作业，阶段和任务
    - 了解某时刻在集群上正在执行什么工作

<h4>Spark日志</h4>

- Spark日志会输出反常事件，或在spark应用程序中有意添加的输出
- Python无法直接与Spark的基于Java的日志记录库集成
- 但是可以使用Python的logging模块或者甚至更简单的打印语句
- 更改Spark的日志级别，只需运行以下命令
    - spark.sparkContext.setLogLevel("INFO")

<h4>Spark UI</h4>

- 每个运行的SparkContext都将启动一个Web UI，默认情况下在端口4040
- 如果运行多个应用程序，它们将各自启动一个Web UI，并累加端口号（4041,4042,4043）
- Spark UI选项卡
    - Jobs选项卡对应Spark作业
    - Stages选项卡对应各个阶段
    - Storage选项卡包含当前在Spark引用程序中缓存的信息和数据
    - Environment选项卡包含有关Spark应用程序的配置等相关信息
    - SQL选项卡对应提交的结构化API查询
    - Executors选项卡提供应用程序的每个执行器的详细信息
- 以下通过一段SQL查询代码，学习使用Spark UI跟踪它的执行情况

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()

In [8]:
spark.read\
    .option("header", "true")\
    .csv("./data/retail-data/all/online-retail-dataset.csv")\
    .repartition(2)\
    .selectExpr("instr(Description, 'GLASS') >= 1 as is_glass")\
    .groupBy("is_glass")\
    .count()\
    .collect()

[Row(is_glass=None, count=1454),
 Row(is_glass=True, count=12861),
 Row(is_glass=False, count=527594)]

- 以下开始分析SQL选项卡，打开 http://localhost:4040/SQL/
- 点击collect查询

- 先回顾一下作业、阶段、任务
- 作业就是由动作操作触发，本例就是由collect方法引出一个作业
- 阶段的转换由shuffle操作触发，一次shuffle导致进入下一个阶段
- 一个数据分区对应一个任务
- 以下展示三个阶段的过程

<h4>第一阶段</h4>

- 读取csv文件，8个分区，我怎么感觉默认是11个分区？

In [12]:
%%html
<img src="img/stage1.png", width=400>

<h4>第二阶段</h4>

- 数据重分区为两个，进入第二阶段
- 同时第二阶段会在每个分区执行groupBy+count
    - 这两个函数，会首先在各个分区内聚合，这不会触发shuffle
    - 两个分区，每个分区输出三行，因此，最终输出行为6行

In [13]:
%%html
<img src="img/stage2.png", width=400>

<h4>第三阶段</h4>

- 将两个分区结果再进行聚合，这时候就需要shuffle

In [15]:
%%html
<img src="img/stage3.png", width=400>

<h4>Jobs选项卡</h4>

- 点开collect这个作业，可以看到三个阶段，分别有11个任务，2个任务和200个任务
- 第一阶段11个任务，这是对输入CSV文件的拆分
- 第二阶段显示调用了重新分区，两个任务
- 第三阶段默认shuffle分区是200个任务

In [18]:
%%html
<img src="img/jobdetail.png", width=800>

<h3>调试和Spark抢救方案</h3>

<h4>Spark作业未启动</h4>

- 表现形式
    - Spark作业无法启动
    - Spark UI不显示集群中其他执行器节点的信息
    - Spark UI报告疑似不正确的信息
- 应对措施
    - 确保节点可以在指定端口上相互通信
    - 确保Spark资源配置正确，避免每个执行器进程被配置太大的内存，而超过了集群管理器的内存资源配额

<h4>执行前错误</h4>

- 表现形式
    - 命令不执行，出现大量错误消息
    - 通过Spark UI看不到任何作业、阶段或任务的运行
- 应对措施
    - 可能是代码错误
    - 确认集群和存储系统间的网络连接情况
    - 可能是库或classpath的路径配置问题

<h4>执行期间错误</h4>

- 表现形式
    - 一个Spark作业在集群上成功运行，但下一个作业失败
    - 多步骤查询中的某个步骤失败
    - 一个已经成功运行过的程序在第二次运行时失败了
    - 难以解析错误信息
- 应对措施
    - 检查数据是否存在，输入格式是否正确
    - 查询的列名拼写错误，或者可能是引用的列、视图或表不存在
    - 用确保格式正确的输入数据来隔离问题，排除数据输入问题后，也可以尝试删除部分代码逻辑

<h4>缓慢任务</h4>

- 表现形式
    - Spark阶段中只剩下少数任务未完成，这些任务运行了很长时间
    - 在Spark UI中可以观察到这些缓慢的任务始终在相同数据集上发生
    - 各阶段都有这些缓慢任务
    - 扩大Spark集群没有太大效果某些执行器进程读取和写入的数据量
- 应对措施
    - 缓慢任务可能是数据不均匀地分布到DataFrame或！RDD分区上导致地
    - 尝试增加分区数
    - 尝试用另一列组合重新分区，比如，如果使用ID列进行分区，若ID是倾斜分布的，那么就会产生缓慢任务
    - 如果列中有许多空值，那么空值都会被分配一个数据分区上
    - 尽可能分配给执行器进程更多的内存
    - 可能有不健康地执行器节点，比如磁盘空间不足
    - 用户定义函数（UDF）在对象分配或业务逻辑中有浪费资源的情况

<h4>缓慢的聚合操作</h4>

- 表现形式
    - 在执行groupby操作时产生缓慢任务
    - 聚合操作之后的作业也执行的非常缓慢
- 应对措施
    - 可能是对数据倾斜的某个key执行了聚合操作
    - 聚合操作前增加分区数量
    - 增加执行器内存配额，这将允许其执行器进程更少地与磁盘交互数据，更快完成任务
    - 分区不均衡，试试用repartition重新随即分区

<h4>缓慢的读写操作</h4>

- 表现形式
    - 从文件分布式系统或外部存储系统上读取数据缓慢
    - 往网络文件系统或blob存储上写入数据缓慢
- 应对措施
    - 开启推测执行（将spark.speculation设置为true）
    - 确保网络连接状态良好
    - 如果在相同节点上运行Spark和HDFS等分布式文件系统，确保Spark与文件系统的节点主机名相同

<h4>驱动器的OutOfMemoryError</h4>

- 表现形式
    - Spark应用程序无响应或崩溃
    - 驱动进程错误日志发现OutOfMemoryErrors错误或垃圾回收消息
    - 命令需要长时间才能完成或根本不运行
    - 驱动器的JVM的内存使用率非常高
- 应对措施
    - 可能是代码尝试使用collect之类的操作将过大的数据集收集到驱动器节点
    - 使用了广播连接，但是广播的数据太大，设置最大广播连接数可以更好地空值广播消息大小
    - 应用程序长时间运行导致驱动器产生大量对象，并且无法释放。
    - 增加驱动器进程内存分配